In [1]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=2, memory_limit='5GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45669,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:33441,Total threads: 2
Dashboard: http://127.0.0.1:34227/status,Memory: 4.66 GiB
Nanny: tcp://127.0.0.1:40539,


In [2]:
from xgboost.dask import DaskXGBClassifier

model = DaskXGBClassifier()
model.load_model("./model_XGB.json")

model.best_iteration

99

In [3]:
import geopandas as gpd
from pystac_client import Client as StacClient
import odc.stac
import planetary_computer as pc

odc.stac.configure_rio(cloud_defaults=True, client=client)

gdf_aoi = gpd.read_file("switzerland.geojson")

catalog = StacClient.open("https://planetarycomputer.microsoft.com/api/stac/v1")

time = ("2020-08-01", "2020-08-31")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=gdf_aoi.geometry[0],
    datetime=time,
    query={
        "eo:cloud_cover": {"lt": 40}
    }
)
items = search.item_collection()
print(f"Found: {len(items)} datasets")

S2_L2A_MEASUREMENTS = ["B02", "B03", "B04", "B05", "B06","B07","B08","B8A","B09", "B11", "B12", "SCL"]

ds = odc.stac.load(
    items,
    bands=S2_L2A_MEASUREMENTS,
    resolution=100,
    dtype='uint16',
    nodata=0,
    resampling='nearest',
    geopolygon=gdf_aoi.__geo_interface__,
    group_by="solar_day",
    fail_on_error=False,
    patch_url=pc.sign,
    chunks={'x': 2048, 'y': 1024, 'time': -1},
)
ds

Found: 96 datasets


<xarray.Dataset> Size: 4GB
Dimensions:      (y: 2213, x: 3495, time: 21)
Coordinates:
  * y            (y) float64 18kB 5.295e+06 5.295e+06 ... 5.074e+06 5.074e+06
  * x            (x) float64 28kB 2.648e+05 2.65e+05 ... 6.142e+05 6.142e+05
    spatial_ref  int32 4B 32632
  * time         (time) datetime64[ns] 168B 2020-08-01T10:05:59.024000 ... 20...
Data variables:
    B02          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B03          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B04          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B05          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B06          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B07          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B08          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B8A          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B09          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B11          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    B12          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>
    SCL          (time, y, x) uint16 325MB dask.array<chunksize=(21, 1024, 2048), meta=np.ndarray>

In [4]:
import numpy as np

# Mask nodata value (0) as np.NaN
ds = ds.where(ds != 0)

# Mask cloudy pixels
ds = ds.where((ds.SCL <= 7) | (ds.SCL == 11))
ds = ds.drop_vars("SCL")

# Median Composition
ds = ds.median(dim='time', skipna=True)
coords = ds.coords

# Convert back to uint16, replacing NaN with 0
ds = ds.fillna(0).astype("uint16")

ds = ds.to_stacked_array("bands", sample_dims=["x","y"])

cols = ds.x.size
rows = ds.y.size
bands = ds.bands.size

ds = ds.data
ds = np.reshape(ds, (rows*cols, bands))
cloud_mask = (ds == 0).any(axis=1)

In [5]:
ds = ds.rechunk({1: -1})
y_pred = model.predict(ds)
y_pred

dask.array<_argmax, shape=(7734435,), dtype=int64, chunksize=(1789440,), chunktype=numpy.ndarray>

In [6]:
import dask

# Option 1

y_pred, cloud_mask = dask.compute(y_pred, cloud_mask)

# Option 2

# y_pred[cloud_mask] = 0
# y_pred = y_pred.compute()

/home/simone/geop-env/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/home/simone/geop-env/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/home/simone/geop-env/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/home/simone/geop-env/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


In [7]:
import numpy as np
import xarray as xr
from odc.geo.xr import write_cog

# Load fitted LabelEncoder
import pickle 
with open('./label_encoder.pkl', 'rb') as f: 
      label_encoder = pickle.load(f)

y_pred = label_encoder.inverse_transform(y_pred).astype("uint8")

y_pred[cloud_mask] = 0
img = np.reshape(y_pred, (rows, cols))

img_da = xr.DataArray(
    data=img,
    coords=coords
)
del img, ds, y_pred

# Write Xarray predictions as COG
write_cog(img_da, "LC_switzerland.tif", nodata=0, overwrite=True)

PosixPath('LC_switzerland.tif')

In [8]:
client.close()